In [1]:
import ee
import geopandas as gpd
import pandas as pd
import pandas as pd
import json

In [2]:
ee.Authenticate()

Enter verification code: 4/1AVHEtk7HU72xl8x9N-FX9eV5qrOt9nThFNSGfnriy0Y6uZghmSl8yK_klZc

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
data = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE");

In [5]:
dataset = data.filterDate('2018-5-1', '2020-5-30')

In [6]:
total = dataset.reduce(ee.Reducer.sum());

In [7]:
ranchi = gpd.read_file("./ranchi/ranchi_new.shp")

In [8]:
js = json.loads(ranchi.to_json())
shpz = ee.Geometry(ee.FeatureCollection(js).geometry())

In [9]:
stats = total.reduceRegion(
  reducer= ee.Reducer.mean(),
  geometry= shpz,
  scale= 5000,
  )

In [10]:
print("Amount = ",stats.get('pr_sum').getInfo())

Amount =  3038.7696670059636


In [11]:
count = total.reduceRegion(
  reducer= ee.Reducer.count(),
  geometry= shpz,
  scale= 5000,
  )

In [12]:
print("Total Pixel = ", count.get('pr_sum').getInfo())

Total Pixel =  215


In [13]:
totalRanchi = total.clip(shpz)
vectors = totalRanchi.sampleRegions(
  collection=shpz,
  geometries= True,
  scale= 5000,
)
vec =(vectors.getInfo())
print(type(vec))
print(vec)

<class 'dict'>
{'type': 'FeatureCollection', 'columns': {}, 'properties': {'band_order': ['aet_sum', 'def_sum', 'pdsi_sum', 'pet_sum', 'pr_sum', 'ro_sum', 'soil_sum', 'srad_sum', 'swe_sum', 'tmmn_sum', 'tmmx_sum', 'vap_sum', 'vpd_sum', 'vs_sum']}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [85.85648327972326, 22.88458186294481]}, 'id': '0_0', 'properties': {'aet_sum': 18808, 'def_sum': 10610, 'pdsi_sum': 4054, 'pet_sum': 29418, 'pr_sum': 3150, 'ro_sum': 1271, 'soil_sum': 28375, 'srad_sum': 51138, 'swe_sum': 0, 'tmmn_sum': 4830, 'tmmx_sum': 7705, 'vap_sum': 50723, 'vpd_sum': 3426, 'vs_sum': 3330}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [85.63190445869338, 22.929497627150784]}, 'id': '0_1', 'properties': {'aet_sum': 18499, 'def_sum': 10726, 'pdsi_sum': 3749, 'pet_sum': 29225, 'pr_sum': 3104, 'ro_sum': 1252, 'soil_sum': 28731, 'srad_sum': 51122, 'swe_sum': 0, 'tmmn_sum': 4759, 'tmmx_sum': 7

In [14]:
pd.json_normalize(vec['properties'])

,band_order
0,"[aet_sum, def_sum, pdsi_sum, pet_sum, pr_sum, ..."


In [15]:
m = pd.json_normalize(vec)

In [16]:
df2 = pd.DataFrame(m)

In [17]:
print(df2)

                type                                           features   
0  FeatureCollection  [{'type': 'Feature', 'geometry': {'geodesic': ...  \

                               properties.band_order  
0  [aet_sum, def_sum, pdsi_sum, pet_sum, pr_sum, ...  


In [55]:
resultz = json.dumps(vec, indent = 3, sort_keys = True)
rio = resultz
lenen = rio.split("\n",2)[2]
lenen_back = '\n'.join(lenen.split('\n')[:-20])
lenin = lenen_back[:-1]
homi = json.dumps(lenin)
print(homi)

"   \"features\": [\n      {\n         \"geometry\": {\n            \"coordinates\": [\n               85.85648327972326,\n               22.88458186294481\n            ],\n            \"geodesic\": false,\n            \"type\": \"Point\"\n         },\n         \"id\": \"0_0\",\n         \"properties\": {\n            \"aet_sum\": 18808,\n            \"def_sum\": 10610,\n            \"pdsi_sum\": 4054,\n            \"pet_sum\": 29418,\n            \"pr_sum\": 3150,\n            \"ro_sum\": 1271,\n            \"soil_sum\": 28375,\n            \"srad_sum\": 51138,\n            \"swe_sum\": 0,\n            \"tmmn_sum\": 4830,\n            \"tmmx_sum\": 7705,\n            \"vap_sum\": 50723,\n            \"vpd_sum\": 3426,\n            \"vs_sum\": 3330\n         },\n         \"type\": \"Feature\"\n      },\n      {\n         \"geometry\": {\n            \"coordinates\": [\n               85.63190445869338,\n               22.929497627150784\n            ],\n            \"geodesic\": false,

In [59]:
dfz = pd.read_json(homi, lines=True,  encoding="utf8", orient='index')

AttributeError: 'list' object has no attribute 'values'

In [58]:
print(dfz)

                                                   0
0     "features": [\n      {\n         "geometry"...
